<a href="https://colab.research.google.com/github/HUSILEBAI/EES-2110/blob/main/EES_2110_Climate_Maps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section 1:

Basic projections

Create maps using [Cartopy](https://scitools.org.uk/cartopy/docs/latest/)

In [ ]:
# Install Cartopy (if not already installed)
!apt-get -qq install python-cartopy python3-cartopy  # Install system dependencies
!pip install cartopy


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Create a new figure with a Cartopy Orthographic projection
plt.figure(figsize=(10, 8))

# Set up the map with Orthographic projection centered at 0°N, 0°E
ax = plt.axes(projection=ccrs.Orthographic(central_longitude=0, central_latitude=30))

# Add coastlines and country borders with higher resolution
ax.coastlines(resolution='110m')
ax.add_feature(cfeature.BORDERS, linestyle=':')

# Add a finer grid (every 10 degrees)
gl = ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
gl.xlocator = plt.MultipleLocator(10)
gl.ylocator = plt.MultipleLocator(10)

# Mask the land with a light gray color
ax.add_feature(cfeature.LAND, facecolor='black')
# Add ocean with a light blue color
# ax.add_feature(cfeature.OCEAN, facecolor='blue')

# Set the title and show the plot
# plt.title('Planet Earth with Fine Grid and Masked Land (Orthographic Projection)')
plt.show()


# Section 2:

Real world data analysis:

## Download climate data

Near surface (2-m) air temperature analysis

Now, let's download the data using command. Here, we use [NCEP/DOE Reanalysis II](https://psl.noaa.gov/data/gridded/data.ncep.reanalysis2.html).

Note that you only need to run this section once. And you can turn it off after you have your data downloaded.

In [ ]:
!curl -O -C - https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis2/Monthlies/gaussian_grid/air.2m.mon.mean.nc


## Read climate data:

Read near surface air temperature with [xarray](https://xarray.dev/)

In [ ]:
file_name = f"air.2m.mon.mean.nc"
t2_dat = xr.open_dataset(file_name)
t2_dat = t2_dat.isel(lat=slice(None, None, -1))
t2_dat

## Extract climate data:

Annual mean near-surface temperature

In [ ]:
lat = t2_dat.lat
lon = t2_dat.lon
air_dat = t2_dat.air
air_dat_subset = air_dat.where((air_dat['time.year'] >= 1980) & (air_dat['time.year'] <= 2023), drop=True)
air_mean = air_dat_subset.mean({'time','level'})
air_mean

## Plot climate data


In [ ]:
import cartopy.util as cutil
from cartopy.feature import NaturalEarthFeature

# Adding cyclic to the data and coordinates
cdata, clon, clat = cutil.add_cyclic(air_mean, lon, lat)
vmin=180
vmax=300
# Plotting
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.Robinson())
ax.set_global()
cb = ax.contourf(clon, clat, cdata,
                 levels=30,
                 vmin = vmin,
                 vmax = vmax,
                 transform=ccrs.PlateCarree(),
                 cmap='hsv')  # Assuming you want to plot the first level
cbar = plt.colorbar(cb, ax=ax, orientation='vertical')
cbar.set_label('Mean temperature [K]')
ax.coastlines()
ax.add_feature(NaturalEarthFeature(category='cultural', name='admin_0_countries', scale='50m', facecolor='none', edgecolor='black'))
plt.title('Annual Mean Temperature (averaged from 1980 - 2023)')
# Tight layout to adjust spacing
plt.tight_layout()
plt.show()